In [28]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch import nn
%matplotlib inline

In [3]:
# reading file
with open('input.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()

In [5]:
len(text)

1115394

In [7]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [9]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [13]:
char_to_idx = {ch:i for i, ch in enumerate(chars)}
idx_to_char = {i:ch for ch, i in char_to_idx.items()}

In [38]:
def encode(x):
    return [char_to_idx(i) for i in x]

def decode(x):
    return ''.join([idx_to_char[i] for i in x])

In [15]:
data = torch.tensor(encode(text), dtype = torch.long)
data.shape

torch.Size([1115394])

In [17]:
n1, n2 = int(len(text) * 0.8), int(len(text) * 0.9)
train_data = data[:n1]
val_data = data[n1:n2]
test_data = data[n2:]

In [32]:
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch("train")
print(xb)
print(yb)

tensor([[ 7, 46, 53, 59, 57, 43, 10,  1],
        [ 1, 49, 47, 52, 45, 42, 53, 51],
        [50, 50,  1, 51, 43,  1, 57, 53],
        [46, 53, 50, 42,  1, 58, 46, 43]])
tensor([[46, 53, 59, 57, 43, 10,  1,  5],
        [49, 47, 52, 45, 42, 53, 51,  1],
        [50,  1, 51, 43,  1, 57, 53,  6],
        [53, 50, 42,  1, 58, 46, 43, 43]])


In [46]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(-1)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim = -1)
            idx_next = torch.multinomial(probs, num_samples = 1)
            idx = torch.cat((idx, idx_next), dim = 1)
        return idx

m = BigramLanguageModel(vocab_size)

idx = torch.zeros((1, 1), dtype = torch.long)
print(decode(m.generate(idx, max_new_tokens = 100)[0].tolist()))


luxbQWOXx,kW 
:C3HZo:rkAIjoDtu
BAHUwYLItFuKW3$:?xoHK!ZqIDuhDzTzwllu-zfG.?D.S$P-iylWll-E
BcyLDwkW.l$o
